**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sun Jun 19 23:20:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 00B3FBA7-E647-4FAB-8F15-382DD1CD55BD.png
 20210430_234951.jpg
'20210519_115254 (1).jpg'
 270236.jpg
'421988_6706174104 Registrasi _ Telkom UniversityBAYU.gdoc'
'421988_6706174104 Registrasi _ Telkom UniversityBAYU.html'
'60524_PENGUJIAN APLIKASI fix.docx'
 6706174104.mp4
 6706174104_testimoni.mp4
 6706174104.zip
 88404CB8-885E-4AF3-B362-EE479F559D3B.png
 aplikasi
'Aplikasi Plafur'
 Artapela
'ASSESSMENT3_D3IF-41-03_BAYU AJI PRAKOSO- ARI WIBOWO-ANDREAS PAKPAHAN-ALDI WAHYU SARAGIH.rar'
 B58865E7-A438-4329-A8C0-F6B1FC389E53.jpeg
 BayuApp.rar
 CitraTes.mp4
 Classroom
 CodeArt.rar
'Colab Notebooks'
 COTS1_6706174104_BAYU_AJI_PRAKOSO.rar
 cots.rar
'D3IF41-03_6706174104_FANTASY (1).blend'
 D3IF41-03_6706174104_FANTASY.blend
 D3RPLA-4103_6706174104.mp4
 D3RPLA-4103_6706174104.rar
 D3RPLA.BAYUAJIPRAKOSO.rar
'DESAIN TUBES ABGD DEVI.rar'
 DiariSaya.rar
 documen
 EB5FB0FC-24D7-4271-A0AF-2B703BCD0B33.jpeg
'FILE MULTIMEDIA.rar'
'fix tata cara.mp4'
'Form Pendaftaran Sebaya.

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15424, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 15424 (delta 0), reused 1 (delta 0), pack-reused 15423
Receiving objects: 100% (15424/15424), 14.03 MiB | 22.62 MiB/s, done.
Resolving deltas: 100% (10365/10365), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-06-19 23:24:24--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  40.5MB/s    in 4.2s    

2022-06-19 23:24:29 (36.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/wenhao-ji-juF599h_uKs-unsplash(1).txt  
  inflating: data/obj/wenhao-ji-juF599h_uKs-unsplash.jpg  
  inflating: data/obj/wenhao-ji-juF599h_uKs-unsplash.txt  
  inflating: data/obj/wild-koala-with-ear-tag-radio-transmitter--1030854506-e6d8ff5cac4343068cd1e708acacfa2c.jpg  
  inflating: data/obj/wild-koala-with-ear-tag-radio-transmitter--1030854506-e6d8ff5cac4343068cd1e708acacfa2c.txt  
  inflating: data/obj/wim-bollen-qvWx2bzsIv8-unsplash.jpg  
  inflating: data/obj/wim-bollen-qvWx2bzsIv8-unsplash.txt  
  inflating: data/obj/zizhang-cheng-tX8M_Xteu4Y-unsplash.jpg  
  inflating: data/obj/zizhang-cheng-tX8M_Xteu4Y-unsplash.txt  
  inflating: data/obj/zizhang-cheng-whGzn3Q45Uw-unsplash.jpg  
  inflating: data/obj/zizhang-cheng-whGzn3Q45Uw-unsplash.txt  
  inflating: data/obj/3PXR46CAMREHJGIBYUUPGBTUE4.jpg  
  inflating: data/obj/3PXR46CAMREHJGIBYUUPGBTUE4.txt  
  inflating: data/obj/5fb8e8551ad37.jpg  
  inflating: data/obj/5fb8e85

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.571429 0.632296 0.428571 0.626459

['15', '0.571429', '0.632296', '0.428571', '0.626459']
0 data/obj/animal-1835689_1280.txt
0 0.571429 0.632296 0.428571 0.626459
15 0.537589 0.500597 0.894988 0.812251

['15', '0.537589', '0.500597', '0.894988', '0.812251']
1 data/obj/dusan-veverkolog-8r4yVl4PFGE-unsplash.txt
0 0.537589 0.500597 0.894988 0.812251
15 0.475233 0.514712 0.745463 0.594743

['15', '0.475233', '0.514712', '0.745463', '0.594743']
2 data/obj/benjamin-sow-_XdTZMuCXRM-unsplash.txt
0 0.475233 0.514712 0.745463 0.594743
15 0.482955 0.561728 0.336777 0.780864

['15', '0.482955', '0.561728', '0.336777', '0.780864']
3 data/obj/ramil-dinapo-pqou5gt20hU-unsplash.txt
0 0.482955 0.561728 0.336777 0.780864
15 0.535218 0.660024 0.614679 0.444751

['15', '0.535218', '0.660024', '0.614679', '0.444751']
4 data/obj/kevin-bosc-oiDC6fDtr3k-unsplash.txt
0 0.535218 0.660024 0.614679 0.444751
15 0.594231 0.530682 0.739744 0.879545

['15', '0.594231', '0.530682', '0.739744', '0.879545']
5 data/

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/jordan-whitt-EerxztHCjM8-unsplash.jpg', 'data/obj/limin-liang-xReE6RgvrVw-unsplash.jpg', 'data/obj/nadir-syzygy-KNpQAX6Q6cU-unsplash.jpg', 'data/obj/Koala-in-tree-photo-credit-Sarah-Scholtes.jpg', 'data/obj/skyler-h-v50ni5LsAsg-unsplash.jpg', 'data/obj/don-ranasinghe-OHqilAOCESY-unsplash.jpg', 'data/obj/craig-garrett-vhTkfZvKbPM-unsplash.jpg', 'data/obj/patrick-mcgregor-emvEcT_sjI8-unsplash.jpg', 'data/obj/dillon-pena-At7EPUv3gKc-unsplash.jpg', 'data/obj/enrico-carcasci-M9lslW1ueLo-unsplash.jpg', 'data/obj/bewimage-tok03a8.jpg', 'data/obj/david-clode-CIMk0FSOrAE-unsplash.jpg', 'data/obj/melody-p-cSNumclQixw-unsplash.jpg', 'data/obj/hugo-kruip-N0HdQGHuY4I-unsplash.jpg', 'data/obj/craig-manners-0RWZ65T3Kxo-unsplash.jpg', 'data/obj/miranda-garside-dNYqZMxWbLQ-unsplash.jpg', 'data/obj/erika-garcia-hxQ7IpzHMoc-unsplash.jpg', 'data/obj/angus-gray-al2BO4QBRRM-unsplash.jpg', 'data/obj/frankie-mish-er-kLJoGevU-unsplash.jpg', 'data/obj/adrian-pereira-8_ZwJSze_IM-unsplash.jpg', 'data/o

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:

# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

/bin/bash: ./darknet: No such file or directory
